In [2]:
from pyspark.sql import SparkSession

In [3]:
saprk = SparkSession.builder.appName("Ship").getOrCreate()

In [4]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [5]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [6]:
from pyspark.ml.feature import StringIndexer  

In [7]:
si =StringIndexer  (inputCol="Cruise_line",outputCol="line")

In [8]:
si = si.fit(df)

In [9]:
si = si.transform(df)

In [10]:
for x in si.head(7):
    print(x)
    print("\n")

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, line=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, line=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, line=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, line=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, line=1.0)


Row(Ship_name='Ecstasy', Cruise_line='Carnival', Age=22, Tonnage=70.367, passengers=20.52, length=8.55, cabins=10.2, passenger_density=34.29, crew=9.2, 

In [11]:
from pyspark.sql.functions import corr

In [12]:
si.select(corr("crew", "length")).show()

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+



In [13]:
si.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'line']

In [14]:
from pyspark.ml.feature import VectorAssembler

In [15]:
vr =VectorAssembler(inputCols=['Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',"line"],outputCol= "Feature")

In [16]:
vr = vr.transform(si)

In [17]:
vr.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|line|             Feature|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|16.0|[30.2769999999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|16.0|[30.2769999999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7| 1.0|[47.262,14.86,7.2...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1| 1.0|[110.0,29.74,9.53...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 1.0|[101.353,26.42,8....|
|    Ecstasy|   

In [18]:
final = vr.select("Feature","crew")

In [19]:
train,test = final.randomSplit([0.7,0.3])

In [20]:
from pyspark.ml.regression import LinearRegression

In [21]:
lr  = LinearRegression(featuresCol='Feature', labelCol='crew')

In [22]:
lr = lr.fit(train)

In [31]:
pre = lr.transform(test)

In [23]:
ev = lr.evaluate(test)

In [24]:
ev.r2

0.8841916713033454

In [25]:
ev.rootMeanSquaredError

1.1798002628138284

In [26]:
train.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               102|
|   mean| 7.507549019607845|
| stddev|3.4902448029761457|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [27]:
unlabel = test.select("Feature")

In [28]:
predict = lr.transform(unlabel)

In [29]:
predict.show()

+--------------------+-------------------+
|             Feature|         prediction|
+--------------------+-------------------+
|[2.329,0.94,2.96,...|0.36514327968415095|
|[12.5,3.94,4.36,0...| 1.2929487275315334|
|[16.8,2.96,5.14,1...|  2.241234964326458|
|[19.093,8.0,5.37,...| 3.4714439254911955|
|[25.0,7.76,6.22,3...|  3.765216380398787|
|[28.0,11.5,6.74,4...|  3.697117245142291|
|[30.2769999999999...| 3.8292631457408204|
|[30.2769999999999...| 3.8292631457408204|
|[30.2769999999999...|  3.434239599640308|
|[30.2769999999999...|  4.070288956469646|
|[34.25,10.52,6.15...|  4.541691469687467|
|[42.0,14.8,7.13,7...|  6.274629972341748|
|[48.563,20.2,6.92...|  5.809731004520749|
|[50.76,17.48,7.54...|  7.092838403274648|
|[53.049,13.44,7.2...|  6.154105557091521|
|[53.872,14.94,7.9...|  6.939976609955304|
|[55.451,12.66,7.1...|  5.889427187330801|
|[58.6,15.66,8.23,...|  7.258359485240918|
|[58.825,15.6,8.23...|  7.146999368463916|
|[61.0,13.8,7.8,6....|  6.519831780388519|
+----------